In [1]:

import geopandas as gpd
import pandas as pd



In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
gdf = gpd.read_file("./data/s1_dv_2019_04_01_2019_10_31.shp", dirver="ESRI Shapefile")
gdf.head()

,date,group_id,relorb,look_dir,system_id,geometry
0,2019-04-01,106_41.89,106,ASCENDING,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190401T22...,"POLYGON ((-77.32291 40.93963, -77.33222 40.970..."
1,2019-04-01,106_43.40,106,ASCENDING,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190401T22...,"POLYGON ((-78.02794 43.61720, -78.11577 43.940..."
2,2019-04-01,106_45.20,106,ASCENDING,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190401T23...,"POLYGON ((-75.48140 46.47212, -75.47715 46.459..."
3,2019-04-04,150_41.77,150,ASCENDING,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190404T23...,"POLYGON ((-83.84993 42.31704, -83.50689 42.365..."
4,2019-04-04,150_43.27,150,ASCENDING,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190404T23...,"POLYGON ((-83.94588 42.67766, -84.06177 43.119..."


In [4]:
gdf['date'] = pd.to_datetime(gdf['date'], format='%Y-%m-%d')
gdf['doy'] = gdf['date'].dt.dayofyear
gdf.head()

,date,group_id,relorb,look_dir,system_id,geometry,doy
0,2019-04-01,106_41.89,106,ASCENDING,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190401T22...,"POLYGON ((-77.32291 40.93963, -77.33222 40.970...",91
1,2019-04-01,106_43.40,106,ASCENDING,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190401T22...,"POLYGON ((-78.02794 43.61720, -78.11577 43.940...",91
2,2019-04-01,106_45.20,106,ASCENDING,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190401T23...,"POLYGON ((-75.48140 46.47212, -75.47715 46.459...",91
3,2019-04-04,150_41.77,150,ASCENDING,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190404T23...,"POLYGON ((-83.84993 42.31704, -83.50689 42.365...",94
4,2019-04-04,150_43.27,150,ASCENDING,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190404T23...,"POLYGON ((-83.94588 42.67766, -84.06177 43.119...",94


In [5]:
def es_mid(gdf): return (181 + 135) // 2
def ls_mid(gdf): return (243 + 182) // 2

gdf = gdf[gdf['relorb'].isin([150, 4, 160, 77])]

gdf = gdf[(gdf['doy'] >= 135) & (gdf['doy'] <= 243)]
gdf['season'] = gdf['doy'].apply(lambda x: 'early' if x <= 181 else 'mid')
gdf['mid_point'] = gdf.apply(lambda x: es_mid(x) if x['season'] == 'early' else ls_mid(x), axis=1)
gdf['diff'] = gdf.apply(lambda x: x['doy'] - x['mid_point'], axis=1)
gdf['diff_abs'] = gdf['diff'].abs()

diffs = gdf.groupby(['group_id', 'relorb', 'season'])['diff_abs'].min().reset_index()



In [6]:
diffs

,group_id,relorb,season,diff_abs
0,150_41.77,150,early,4
1,150_41.77,150,mid,2
2,150_43.27,150,early,4
3,150_43.27,150,mid,2
4,150_44.74,150,early,4
5,150_44.74,150,mid,2
6,150_45.75,150,early,2
7,150_45.75,150,mid,4
8,4_41.19,4,early,2
9,4_41.19,4,mid,4


In [7]:
data = {k: [] for k in gdf.columns}
gdfr = gpd.GeoDataFrame(data)
gdfr['geometry'] = gpd.GeoSeries()
gdfr

,date,group_id,relorb,look_dir,system_id,geometry,doy,season,mid_point,diff,diff_abs


In [8]:
gdf[(gdf['group_id'] == '150_41.77') & (gdf['season'] == 'early') & (gdf['diff_abs'] == 4 )]

,date,group_id,relorb,look_dir,system_id,geometry,doy,season,mid_point,diff,diff_abs
73,2019-06-03,150_41.77,150,ASCENDING,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190603T23...,"POLYGON ((-80.83548 42.70620, -80.83295 42.706...",154,early,158,-4,4


In [9]:
frames = []
gdf2 = gdf.copy()
for idx, d in diffs.iterrows():
    frames.append(gdf2[(gdf2['group_id'] == d['group_id']) & (gdf2['season'] == d['season']) & (gdf2['diff_abs'] == d['diff_abs'] )])

out = pd.concat(frames, ignore_index=True)


In [10]:
out['date'] = pd.to_datetime(out['date']).dt.strftime('%Y-%m-%d')
out.to_file("./data/output.shp", driver='ESRI Shapefile')